Blah de blah

In [65]:
# This tool will not ensure that pixels and boundaries align
# That is the job of the user
# Must be operating as dev account
import pandas
import numpy
from osgeo import gdal, gdalconst
from osgeo import ogr
import gc
import tempfile
import datetime
import dateutil
import time
import os
import sys
import ipyparallel as ipp
#from joblib import Parallel, delayed
#import multiprocessing
sys.path.append('..\\Modules')
#import usle_cfactor_analysis
#import zonal_stats_ForLayerObject
import resample_raster
#from rasterstats import zonal_stats
#import matplotlib.pyplot as plt
#%matplotlib inline

#Path to intersected Subcatch/FUs shapefile
pathToShapefile = r"P:\projects\ReefProgrammeData1920\Grazing\BU\BU_IntSCFU2019_GrzChgOnly.shp"
#pathToShapefile = r"P:\projects\ReefProgrammeData1920\Grazing\BU\TestData\BU_IntSCFU2019_SC1386.shp"

#parentCSVOutputDir = r"P:\projects\ReefProgrammeData1920\Grazing\BU\TestData\Python\ChangeFiles1386"
#parentCSVOutputDir = r"P:\projects\ReefProgrammeData1920\Grazing\BU\TestData\Python\ChangeFiles"
parentCSVOutputDir = r"P:\projects\ReefProgrammeData1920\Grazing\BU\TestData\Python\BaselineFiles"

uniqueIDField = "IntSCFU"
catchmentField = "IntSCs"
FUField = "IntFUs"

FUsToInclude = ["Grazing Open","Grazing Forested"]

#Path to template raster (probably subcatchments or DEM)
rasterTemplateFile = r"P:\projects\Reefscape\SRTM_H_1SEC_30M_REGIONAL\BU\final_network_creation\bucatch30m.asc"
#rasterTemplateFile = r"P:\projects\ReefProgrammeData1920\Grazing\BU\TestData\busc1386.tif"

#Output intersected Subcat/FU raster
#Will get a CSV of same name written 'mapping' each FU to a unique ID (the unique ID will be the raster values)
#rasterOutputFile = r"P:\projects\ReefProgrammeData1920\Grazing\BU\TestData\Python\BU_USLE_IntSubCatFu.tif"
rasterOutputFile = os.path.join(tempfile.gettempdir(), "BU_USLE_IntSubCatFu.tif")

#Path to K raster
rasterKFile = r"P:\projects\ReefProgrammeData1920\Grazing\BU\TestData\ksr.asc"
#rasterKFile = r"P:\projects\ReefProgrammeData1920\Grazing\BU\TestData\ksr1386.tif"

##Path to LS raster
#rasterLSFile =r"C:\DDrive\MW\v450projs\Ram\mw_ls_fact.tif"

#Path to Fines raster
rasterFinesFile = r"P:\projects\ReefProgrammeData1920\Grazing\BU\TestData\cssurf.asc"
#rasterFinesFile = r"P:\projects\ReefProgrammeData1920\Grazing\BU\TestData\cssurf1386.tif"

#Path to Scald raster
#rasterScaldFile = r"C:\DDrive\MW\v450projs\Ram\Exported Catchments Raster.asc"

#Path to Cfactor directory
cFactorDIR = r"P:\projects\ReefProgrammeData1920\Grazing\BU\TestData\CFactor"
#cFactorDIR = r"P:\projects\ReefProgrammeData1920\Grazing\BU\TestData\OneCFactor"

#Path to C-factor Change Mask shapefile
performChange = False
pathToChangeMask = r"P:\projects\ReefProgrammeData1920\Grazing\BU\BU_RC11_Cfact_change_mask1920.shp"
#pathToChangeMask = r"P:\projects\ReefProgrammeData1920\Grazing\BU\TestData\BU_RC11_ChgMask_SC1386.shp"
#pathToChangeRasterOutputs = r"P:\projects\ReefProgrammeData1920\Grazing\BU\TestData\Python"
ParamA = "NonLinearA"
ParamB = "NonLinearB"

#Cfactor extension
cFactExt = "tif"

#cFactor Date naming pattern
cFactPattern = "xxxyyyymm"

monthToSeasonOffsets = {1:-1, 2:-2, 3:0, 4:-1, 5:-2, 6:0, 7:-1, 8:-2, 9:0, 10:-1, 11:-2, 12:0}

#More Cfact settings
defaultCfact = 0.05
reqdValidProp = 0.20

#declare a couple of variables
IntSCFUStr = "IntSCFU"
MeanKLSCStr = "Mean_KLSC"
MeanKLSCFinesStr = "Mean_KLSC_Fines"
MeanKStr = "Mean_K"
MeanFinesStr = "Mean_Fines"
MeanCfactStr = "Mean_CFact"
MeanLSStr = "Mean_LS"
MeanScaldStr = "Mean_Scald"
AreaStr = "Area"
TimeStampStr = "Time_Stamp"
CellCountStr = "Cell_Count"

#Directory and filename constants for USLE parameterisation
USLE_DIR_KLSC = "KLSC";
USLE_DIR_KLSCFINE = "KLSC_Fines";
USLE_DIR_KLSC_CFACT = "Cfact";
USLE_FILE_KLSC = "USLE_KLSC_Total";
USLE_FILE_KLSCFINES = "USLE_KLSC_FinePerc";
USLE_FILE_KLS_CFACT = "CFactor";

rasterResultsDF = pandas.DataFrame(columns=[IntSCFUStr,TimeStampStr,MeanKLSCStr,MeanKLSCFinesStr,MeanKStr,MeanCfactStr,MeanFinesStr,MeanLSStr,MeanScaldStr,AreaStr,CellCountStr])
#rasterResultsDF

csvDirs = ['CFact','KLSC','KLSC_Fines']

In [66]:
if not os.path.exists(parentCSVOutputDir):
    os.makedirs(parentCSVOutputDir)
    print("Made dir: " + parentCSVOutputDir)

for directory in csvDirs:
    if not os.path.exists(parentCSVOutputDir + '\\' + directory):
        os.makedirs(parentCSVOutputDir + '\\' + directory)
        print("Made dir: " + parentCSVOutputDir + '\\' + directory)

Made dir: P:\projects\ReefProgrammeData1920\Grazing\BU\TestData\Python\BaselineFiles
Made dir: P:\projects\ReefProgrammeData1920\Grazing\BU\TestData\Python\BaselineFiles\CFact
Made dir: P:\projects\ReefProgrammeData1920\Grazing\BU\TestData\Python\BaselineFiles\KLSC
Made dir: P:\projects\ReefProgrammeData1920\Grazing\BU\TestData\Python\BaselineFiles\KLSC_Fines


In [67]:
### as dev.ellisrj, start ipcluster in Anaconda prompt:
###> ipcluster start -n 6

rc = ipp.Client()
ar = rc[:].apply_async(os.getpid)
pid_map = ar.get_dict()
pid_map

{0: 16724, 1: 16284, 2: 7676, 3: 18896, 4: 5480}

In [68]:
#load the template raster
templateRas = gdal.Open(rasterTemplateFile)
template_GeoTrans = templateRas.GetGeoTransform()
template_GeoTrans
template_Projection = templateRas.GetProjection()
template_Projection

'PROJCS["unnamed",PROJECTION["Albers_Conic_Equal_Area"],PARAMETER["standard_parallel_1",-18],PARAMETER["standard_parallel_2",-36],PARAMETER["latitude_of_center",0],PARAMETER["longitude_of_center",132],PARAMETER["false_easting",0],PARAMETER["false_northing",0],UNIT["METERS",1]]'

In [69]:
#Load K factor
kFactRas = gdal.Open(rasterKFile)
kfact_GeoTrans = kFactRas.GetGeoTransform()
kfact_GeoTrans

(1275658.63051, 30.0, 0.0, -2008360.4690071, 0.0, -30.0)

In [70]:
if (kfact_GeoTrans == template_GeoTrans):
    print("Kfactor raster resolution info matches template, excellent")
else:
    print("Kfactor raster resolution info didn't match template, attempted a resample")
    kFactRas = resample_raster.resample_raster_to_match_template(templateRas, kFactRas, os.path.join(tempfile.gettempdir(), "resampledKFact.tif"))
    #kFactRas = resample_raster_to_match_template(templateRas, kFactRas, "D:/aa/tester.tif")
    kfact_GeoTrans = kFactRas.GetGeoTransform()

Kfactor raster resolution info didn't match template, attempted a resample


In [71]:
if (kfact_GeoTrans == template_GeoTrans):
    print("Kfactor raster resolution info now matches template, excellent")
    #kFactRas = None
    #kFactRas = gdal.Open(os.path.join(tempfile.gettempdir(), "resampledKFact.tif"))
    #kfact_GeoTrans = kFactRas.GetGeoTransform()
else:
    print("Kfactor raster resolution still not the same as the template, you shouldn't proceed")
#zone_ds = None
#print("rows: " + str(kFactRas.RasterYSize) + " cols: " + str(kFactRas.RasterXSize))

Kfactor raster resolution info now matches template, excellent


In [72]:
#kFactRas.GetRasterBand(1).GetNoDataValue()
templateRas.GetRasterBand(1).GetNoDataValue()

# The line below would get the kfactor raster written to disk, but it may not be necessary
#kFactRas = None

#this stuf to draw up a sample of KFactor, have to chop off heaps of edges so that the range in the colorbar doesn't consider -9999
#kFactRasArray = kFactRas.ReadAsArray()
#array_chop = kFactRasArray[3500:-3500,1500:-1500]  # chop off the outer specified pixels
#plt.imshow(array_chop, interpolation='nearest')
#plt.colorbar()
#plt.title('K Factor')
#plt.show()

-9999.0

In [73]:
##Load LS factor
#lsFactRas = gdal.Open(rasterLSFile)
#lsFact_GeoTrans = lsFactRas.GetGeoTransform()


In [74]:
#if (lsFact_GeoTrans == template_GeoTrans):
#    print("LSfactor raster resolution info matches template, excellent")
#else:
#    print("LSfactor raster resolution info didn't match template, attempted a resample")
#    lsFactRas = resample_raster.resample_raster_to_match_template(templateRas, lsFactRas, os.path.join(tempfile.gettempdir(), "resampledLSFact.tif"))
#    lsFact_GeoTrans = lsFactRas.GetGeoTransform()

In [75]:
#if (lsFact_GeoTrans == template_GeoTrans):
#    print("LSfactor raster resolution info now matches template, excellent")
#    #lsFactRas = None
#    #lsFactRas = gdal.Open(os.path.join(tempfile.gettempdir(), "resampledLSFact.tif"))
#    #lsfact_GeoTrans = lsFactRas.GetGeoTransform()
#else:
#    print("LSfactor raster resolution still not the same as the template, you shouldn't proceed")

In [76]:
#Load Fines raster
FinesRas = gdal.Open(rasterFinesFile)
fines_GeoTrans = FinesRas.GetGeoTransform()
fines_GeoTrans

(1275658.63051, 30.0, 0.0, -2008360.4690071, 0.0, -30.0)

In [77]:
if (fines_GeoTrans == template_GeoTrans):
    print("Fines raster resolution info matches template, excellent")
else:
    print("Fines raster resolution info didn't match template, attempted a resample")
    FinesRas = resample_raster.resample_raster_to_match_template(templateRas, FinesRas, os.path.join(tempfile.gettempdir(), "resampledFines.tif"))
    fines_GeoTrans = FinesRas.GetGeoTransform()

Fines raster resolution info didn't match template, attempted a resample


In [78]:
if (fines_GeoTrans == template_GeoTrans):
    print("Fines raster resolution info now matches template, excellent")
    #FinesRas = None
    #FinesRas = gdal.Open(os.path.join(tempfile.gettempdir(), "resampledFines.tif"))
    #fines_GeoTrans = FinesRas.GetGeoTransform()
else:
    print("Fines raster resolution info still not the same as the template, you shouldn't proceed")

Fines raster resolution info now matches template, excellent


In [79]:
#Open Int SCFU shapefile
basePolys = ogr.Open(pathToShapefile)
type(basePolys)
nlay = basePolys.GetLayerCount()
lyr = basePolys.GetLayer(0)
ext = lyr.GetExtent()
#nlay
#ext
lyr.GetFeatureCount()
#plt.imshow(lyr)

266

In [80]:
featCount = 0
for feat in lyr:
    ##print("Doing something")
    #print(feat.GetFieldAsString(catchmentField) + " and FU " + feat.GetFieldAsString(FUField))
    featCount += 1
    if (featCount == 20):
        break

In [81]:
#FUsToInclude = ["Grazing Open","Grazing Forested","Forestry"]

FUsAsSQL = ""
count = 0
for thisFU in FUsToInclude:
    if(count == 0):
        FUsAsSQL = "'" + thisFU + "'"
    else:
        FUsAsSQL += ",'" + thisFU + "'"
    count += 1

FUsAsSQL

"'Grazing Open','Grazing Forested'"

In [82]:
lyr.SetAttributeFilter( FUField + " IN (" + FUsAsSQL + ")")
lyr.GetFeatureCount()

266

In [83]:
csvFile = rasterOutputFile.replace(".tif",".csv")
fumapper = pandas.DataFrame(columns=[uniqueIDField, catchmentField, FUField])
#featCount = 0
for feat in lyr:
    #print(feat.GetFieldAsString(catchmentField) + " and FU " + feat.GetFieldAsString(FUField))
    #fumapper.append([feat.GetFieldAsString(uniqueIDField),feat.GetFieldAsString(catchmentField),feat.GetFieldAsString(FUField)])
    newList = [feat.GetFieldAsString(uniqueIDField),feat.GetFieldAsString(catchmentField),feat.GetFieldAsString(FUField)]
    fumapper.loc[len(fumapper)] = newList
    #featCount += 1
    #if (featCount == 20):
    #    break

#lyr.GetFeatureCount()
#fumapper.to_csv(csvFile, index=False)
#fumapper

In [84]:
##help(templateRas)
geo_transform = templateRas.GetGeoTransform()
x_min = geo_transform[0]
y_max = geo_transform[3]
x_max = x_min + geo_transform[1] * templateRas.RasterXSize
y_min = y_max + geo_transform[5] * templateRas.RasterYSize
x_count = templateRas.RasterXSize
y_count = templateRas.RasterYSize
pixel_width = geo_transform[1]
pixel_height = geo_transform[5]

In [85]:
####Original
zone_ds = gdal.GetDriverByName("GTiff").Create(rasterOutputFile, x_count, y_count, 1, gdal.GDT_Int32)
##Doing MEM did not yield the expected unique vals
##zone_ds = gdal.GetDriverByName("MEM").Create("", x_count, y_count, 1, gdal.GDT_Byte)
# This was shifting the resulting raster northwards in our Albers stuff
#zone_ds.SetGeoTransform((x_min, pixel_width, 0, y_min, 0, pixel_width))
zone_ds.SetGeoTransform((x_min, pixel_width, 0, y_max, 0, pixel_height))

##Do not set to Aust Albers... missing PROJ4 details means that this rasterization fails
#zone_ds.SetProjection(template_Projection)
band = zone_ds.GetRasterBand(1)
NoData_value = -9999
band.SetNoDataValue(NoData_value)
band.FlushCache()
gdal.RasterizeLayer(zone_ds, [1], lyr, options=["ATTRIBUTE=" + uniqueIDField])
zonesRasterArray = zone_ds.ReadAsArray()
#zoneMask = numpy.ma.masked_where(zonesRasterArray == NoData_value, zonesRasterArray)

In [86]:
# NoData_value = -9999
# zone_ds = gdal.GetDriverByName("GTiff").Create(rasterOutputFile, x_count, y_count, 1, gdal.GDT_Int32)
# #zone_ds = gdal.GetDriverByName("MEM").Create("", x_count, y_count, 1, gdal.GDT_Int32)
# gdal.RasterizeLayer(zone_ds, [1], lyr, options=["ATTRIBUTE=" + uniqueIDField])
# ##Doing MEM did not yield the expected unique vals
# ##zone_ds = gdal.GetDriverByName("MEM").Create("", x_count, y_count, 1, gdal.GDT_Byte)
# # This was shifting the resulting raster northwards in our Albers stuff
# #zone_ds.SetGeoTransform((x_min, pixel_width, 0, y_min, 0, pixel_width))
# zone_ds.SetGeoTransform((x_min, pixel_width, 0, y_max, 0, pixel_height))
# #band = zone_ds.GetRasterBand(1)
# zonesRasterArray = zone_ds.ReadAsArray()
# ##band.WriteArray(zonesFilled)
# #band.FlushCache()
# ##NoData_value = -9999
# #band.SetNoDataValue(NoData_value)

# zone_ds.GetRasterBand(1).FlushCache()
# zone_ds.GetRasterBand(1).SetNoDataValue(NoData_value)

# #zone_ds = None


In [87]:
#zone_ds.GetGeoTransform()

In [88]:
#NoData_value = -9999
#zonesRasterArray = zone_ds.ReadAsArray()
##zonesFilled = zonesRasterArray.filled(NoData_value)
##zoneMask = numpy.ma.masked_values(zonesFilled, NoData_value)
#zoneMask = numpy.ma.masked_where(zonesRasterArray == NoData_value, zonesRasterArray)
#zonesFilled = zoneMask.filled(NoData_value)
###uniq = numpy.unique(zonesRasterArray)
####print(uniq)

#band.WriteArray(zonesFilled)
#band.FlushCache()
##NoData_value = -9999
#band.SetNoDataValue(NoData_value)


In [89]:
###Change Mask

paramAmask = None
paramBmask = None

if performChange:
    print("Making Change Rasters")
    changePolys = ogr.Open(pathToChangeMask)
    type(basePolys)
    changenlay = changePolys.GetLayerCount()
    changelyr = changePolys.GetLayer(0)
    changeext = changelyr.GetExtent()
    #changenlay
    #changeext
    changelyr.GetFeatureCount()
    
    changegeo_transform = templateRas.GetGeoTransform()
    changex_min = changegeo_transform[0]
    changey_max = changegeo_transform[3]
    changex_max = changex_min + changegeo_transform[1] * templateRas.RasterXSize
    changey_min = changey_max + changegeo_transform[5] * templateRas.RasterYSize
    changex_count = templateRas.RasterXSize
    changey_count = templateRas.RasterYSize
    changepixel_width = changegeo_transform[1]
    changepixel_height = changegeo_transform[5]
    
    NoData_value = -9999
    
    change_A_ds = gdal.GetDriverByName("GTiff").Create(os.path.join(tempfile.gettempdir(), ParamA + ".tif"),
                                                       changex_count, changey_count, 1, gdal.GDT_Float32)
    #change_A_ds = gdal.GetDriverByName("GTiff").Create("MEM", changex_count, changey_count, 1, gdal.GDT_Float32)
    change_A_ds.SetGeoTransform((changex_min, changepixel_width, 0, changey_max, 0, changepixel_height))
    #change_A_ds.SetProjection(template_Projection)
    #change_A_band = change_A_ds.GetRasterBand(1)    
    #change_A_band.SetNoDataValue(NoData_value)
    change_A_ds.GetRasterBand(1).SetNoDataValue(NoData_value)
    #change_A_band.FlushCache()
    change_A_ds.GetRasterBand(1).FlushCache()
    gdal.RasterizeLayer(change_A_ds, [1], changelyr, options=["ATTRIBUTE=" + ParamA])
    
    change_B_ds = gdal.GetDriverByName("GTiff").Create(os.path.join(tempfile.gettempdir(), ParamB + ".tif"),
                                                       changex_count, changey_count, 1, gdal.GDT_Float32)
    #change_B_ds = gdal.GetDriverByName("GTiff").Create("MEM", changex_count, changey_count, 1, gdal.GDT_Float32)
    change_B_ds.SetGeoTransform((changex_min, changepixel_width, 0, changey_max, 0, changepixel_height))
    #change_B_ds.SetProjection(template_Projection)
    #change_B_band = change_B_ds.GetRasterBand(1)    
    #change_B_band.SetNoDataValue(NoData_value)
    change_B_ds.GetRasterBand(1).SetNoDataValue(NoData_value)
    #change_B_band.FlushCache()
    change_B_ds.GetRasterBand(1).FlushCache()
    gdal.RasterizeLayer(change_B_ds, [1], changelyr, options=["ATTRIBUTE=" + ParamB])
    
    print("Made Change Rasters, converting to Numpy arrays")
    paramAnumpy = change_A_ds.ReadAsArray()
    paramBnumpy = change_B_ds.ReadAsArray()
    
    paramAmask = numpy.ma.masked_where(paramAnumpy == NoData_value, paramAnumpy)
    paramBmask = numpy.ma.masked_where(paramBnumpy == NoData_value, paramBnumpy)
    
    change_A_ds = None
    change_B_ds = None
    
    paramAnumpy = None
    paramBnumpy = None
    
    print("Masked change Numpy arrays ready to go")

In [90]:
#now need to combine K, LS & Fines

kFactNumpy = kFactRas.GetRasterBand(1).ReadAsArray()
#lsFactNumpy = lsFactRas.GetRasterBand(1).ReadAsArray()
FinesNumpy = FinesRas.GetRasterBand(1).ReadAsArray()

kFactMasked = numpy.ma.masked_where(kFactNumpy == kFactRas.GetRasterBand(1).GetNoDataValue(), kFactNumpy)
#lsFactMasked = numpy.ma.masked_where(lsFactNumpy == lsFactRas.GetRasterBand(1).GetNoDataValue(), lsFactNumpy)
FinesMasked = numpy.ma.masked_where(FinesNumpy == FinesRas.GetRasterBand(1).GetNoDataValue(), FinesNumpy)



kFactRas = None
#lsFactRas = None
FinesRas = None




In [91]:
#what will this do for nodata?
# it will give -9999 ^ 3. Need to mask!
#KLSFinesNumpy = kFactMasked * lsFactMasked * (FinesMasked / 100)
#KLSNumpy = kFactMasked * lsFactMasked
KLSFinesNumpy = kFactMasked * (FinesMasked / 100)
KLSNumpy = kFactMasked
print(KLSFinesNumpy.min())
#print(KLSFinesNumpy.max())


2.18205e-06


In [92]:
fines_GeoTrans

(1270978.63051, 30.0, 0.0, -2004160.4690071, 0.0, -30.0)

In [93]:
XCoordMid = fines_GeoTrans[0] + (25 * fines_GeoTrans[1]) + (fines_GeoTrans[1] / 2)
XCoordMid 

1271743.63051

In [94]:
YCoordMid = fines_GeoTrans[3] + (25 * fines_GeoTrans[5]) + (fines_GeoTrans[5] / 2)
YCoordMid

-2004925.4690071

In [95]:
theRowForX = (1678633.63051 - fines_GeoTrans[0] - (fines_GeoTrans[1] / 2)) / fines_GeoTrans[1]
theRowForX

13588.0

In [96]:
theColForY = (-2258725.4690071 - fines_GeoTrans[3] - (fines_GeoTrans[5] / 2)) / fines_GeoTrans[5]
theColForY

8485.0

In [97]:
def centreCoordsForPixel(gt, row, col):
    
    XCoordMid = gt[0] + (col * gt[1]) + (gt[1] / 2)
    YCoordMid = gt[3] + (row * gt[5]) + (gt[5] / 2)
    
    return (XCoordMid, YCoordMid)

In [98]:
def pixelForCentreCoords(gt, xCoord, yCoord):
    
    yCol = int((xCoord - gt[0] - (gt[1] / 2)) / gt[1])
    xRow = int((yCoord - gt[3] - (gt[5] / 2)) / gt[5])
    
    return(xRow, yCol)

In [99]:
tester = pixelForCentreCoords(fines_GeoTrans, 1678633.63051, -2258725.4690071)
tester

(8485, 13588)

In [100]:
test2 = centreCoordsForPixel(fines_GeoTrans, 33, 66)
test2[1]

-2005165.4690071

In [101]:
zone_ds.GetGeoTransform()


(1270978.63051, 30.0, 0.0, -2004160.4690071, 0.0, -30.0)

In [102]:

def getDateFromFileName(theFileName, pattern):
    
    allowedChars = ["x", "y", "m", "d"]
    
    for cha in pattern:
        if cha not in allowedChars:
            print("You can't use '" + cha + "' ya dill")
            return
    
    
    if "yyyy" not in pattern:
        print("You've got to at least have 'yyyy' in your pattern")
        return
    
    Day = 1
    Month = 1
    Year = int(theFileName[pattern.index("yyyy"): pattern.index("yyyy") + 4])
    
    if "mm" in pattern:
        Month = int(theFileName[pattern.index("mm"): pattern.index("mm") + 2])
    
   
    if "dd" in pattern:
        Day = int(theFileName[pattern.index("dd"): pattern.index("dd") + 2])
        

    boogity = datetime.date(Year, Month, Day)
    
    return boogity

In [103]:
# list out Cfactor files
#cFactorDIR = "D:\MW\Ram_USLE\skip"
#cFactorDIR = r"P:\projects\ReefProgrammeData1920\Grazing\BU\TestData\CFactor"

cRastersInfo = {}
cRastersAlone = []
for file in os.listdir(cFactorDIR):
    if file.endswith("." + cFactExt):
        #print(os.path.join(cFactorDIR, file) + " and " + str(getDateFromFileName(file, "xxxyyyymm")))
        #cRastersInfo.append(getDateFromFileName(file, "xxxyyyymm"), os.path.join(cFactorDIR, file))
        cRastersInfo[getDateFromFileName(file, cFactPattern)] =  os.path.join(cFactorDIR, file)
        cRastersAlone.append(os.path.join(cFactorDIR, file))

#cRastersInfo = sorted(cRastersInfo, reverse=True)
cRastersInfoKeysSorted = sorted(cRastersInfo)
#print(cRastersInfoKeysSorted)

In [104]:
#checkdate = cRastersInfoKeysSorted[0]

#print(cRastersInfo.get(checkdate))

#print(str(zonesRasterArray[100,100]))
#print(str(KLSNumpy[100,100]))
#print(str(zonesRasterArray[1000,1000]))
#print(str(NoData_value))

#Need to re-fill the masked KLS arrays so that we can detect 'nodata' cells
#mostly where our coastal mask is missing a slope cell or two
KLSNumpyFilled = numpy.ma.filled(KLSNumpy, NoData_value)
KLSFinesNumpyFilled = numpy.ma.filled(KLSFinesNumpy, NoData_value)

paramAFilled = None
paramBFilled = None

if performChange:
    paramAFilled = numpy.ma.filled(paramAmask, NoData_value)
    paramBFilled = numpy.ma.filled(paramBmask, NoData_value)

In [105]:
#uniqA = numpy.unique(paramAmask)
#print(uniqA)

In [106]:
zoneIDStr = "zoneID"
zoneCountStr = "zoneCount"
validKLSCSum = "validKLSCSum"
allKLSCSum = "allKLSCSum"
validKLSCFinesSum = "validKLSCFinesSum"
allKLSCFinesSum = "allKLSCFinesSum"
validDataCountStr = "dataCount"

validCFactSum = "validCFactSum"
allCFactSum = "allCFactSum"

KLSCValStr = "KLSC"
KLSCFinesValStr = "KLSCFines"
CFactValStr = "CFact"

def analyseThisCFactorRaster(theCFactorRas, datePattern, theZonesRasterArray, zonesGT, 
                             ZonesRowCount, ZonesColCount, ZoneND, theDefaultC, theReqValidProp, 
                             theKLSNumpy, theKLSFinesNumpy, paramANumpy, paramBNumpy, localPerformChange):
    # do pixel by pixel analysis
    # zones raster of intersected FUs will exist, and KLSFines raster will be co-located with this
    # send back a dictionary that also includes the valid/invalid cFactor numbers for each FU
    # should be able to access rasters etc without passing them into this function
    # same for the dictionary of results, it could already exist
    
    # Will need to watch the Y min/max interpretation for our Aus Albers stuff
    zoneIDStr = "zoneID"
    zoneCountStr = "zoneCount"
    validKLSCSum = "validKLSCSum"
    allKLSCSum = "allKLSCSum"
    validKLSCFinesSum = "validKLSCFinesSum"
    validCFactSum = "validCFactSum"
    allKLSCFinesSum = "allKLSCFinesSum"
    allCFactSum = "allCFactSum"
    validDataCountStr = "dataCount"
    KLSCValStr = "KLSC"
    KLSCFinesValStr = "KLSCFines"
    CFactValStr = "CFact"
    
    theRepDate = getDateFromFileName(os.path.basename(theCFactorRas), datePattern)
    #print("TheDate: " + str(theRepDate))
    #print("Do Change: " + str(localPerformChange))
    
        
    cFactRas = gdal.Open(theCFactorRas)
    cFact_GeoTrans = cFactRas.GetGeoTransform()
    #print(cFact_GeoTrans)
    cFactND = cFactRas.GetRasterBand(1).GetNoDataValue()
    cFactNumpy = cFactRas.ReadAsArray()
    
    #print("CFactND: " + str(cFactND))
    
    cFactRows = cFactRas.RasterYSize
    cFactCols = cFactRas.RasterXSize
    
    #zGT = theZones.GetGeoTransform()
    zGT = zonesGT
    
    #theZonesRasterArray = theZones.ReadAsArray()
    

    statsDict = {}
    statsDictToReturn = {}
    uniq = numpy.unique(theZonesRasterArray)
    for num in uniq:
        #using masked, so this check for NoData should be redundant
        #print("trying this UID: " + str(num))
        if num == ZoneND:
            continue
        
        statsDict[num] = {zoneCountStr: int(0), validKLSCSum: float(0), validKLSCFinesSum: float(0), validCFactSum: float(0),
                          allKLSCSum: float(0), allKLSCFinesSum: float(0), allCFactSum: float(0), validDataCountStr: int(0)}

    rowid = 0
    while rowid < ZonesRowCount:
    #while rowid < theZones.RasterYSize:
        #reset colid
        colid = 0
        #if rowid % 500 == 0:
        #    print("Starting row: " + str(rowid))
        
        while colid < ZonesColCount:
        #while colid < theZones.RasterXSize:
            zoneVal = theZonesRasterArray[rowid, colid]
            
            if zoneVal == ZoneND:
                colid+=1
                continue
            
            #we have explicitly ensured that KLSFines matches our pixels
            #although we will have filled the mask with NoData vals
            #so just need to get the requisite cFactor row/col
            
            KLS = theKLSNumpy[rowid, colid]
            if KLS == ZoneND:
                colid+=1
                continue
            
            KLSFines = theKLSFinesNumpy[rowid, colid]
            if KLSFines == ZoneND:
                colid+=1
                continue
            
            
            zoneCoords = centreCoordsForPixel(zGT, rowid, colid)
            #print("Zone Coords: " + str(zoneCoords))
            cFactPix = pixelForCentreCoords(cFact_GeoTrans, zoneCoords[0], zoneCoords[1])
            #print("cFactPix: " + str(cFactPix))
            
            #if cFactPix[0] >= cFactRows:
            #    print("Exceeded row count at " + str(cFactPix[0]) + " for coords: " + str(zoneCoords)
            #          + " where zone pixels were: " + str(rowid) + ", " + str(colid))
            #    SystemExit(0)
            
            #if cFactPix[1] >= cFactCols:
            #    print("Exceeded col count " + str(cFactPix[1]) + " for coords: " + str(zoneCoords)
            #          + " where zone pixels were: " + str(rowid) + ", " + str(colid))
            #    SystemExit(0)
            
            theC = theDefaultC
            
            isValid = False
            cRasVal = cFactNumpy[cFactPix[0], cFactPix[1]]
            if not cRasVal == cFactND:
                theC = cRasVal
                isValid = True
            
            #Check for change mask....
            if localPerformChange:
                parA = paramANumpy[rowid, colid]
                parB = paramBNumpy[rowid, colid]
                
                #print("Par A: " + str(parA))
                #print("Par B: " + str(parB))
                
                if not parA == ZoneND:
                    potNewC = parA * (theC ** parB)
                    
                    if potNewC < theC and potNewC > 0:
                        theC = potNewC
            
            KLSC = KLS * theC
            KLSCFines = KLSFines * theC
            
            #if(KLSC < 0):
            #    print("Badness at "  + str(rowid) + ", " + str(colid) + " where KLS was: " 
            #          + str(theKLSNumpy[rowid, colid]) + " and C was: " + str(theC))
            #    SystemExit(0)
            
            statsDict[zoneVal][zoneCountStr] += 1
            statsDict[zoneVal][allKLSCSum] += KLSC
            statsDict[zoneVal][allKLSCFinesSum] += KLSCFines
            statsDict[zoneVal][allCFactSum] += theC
            
            if(isValid):
                statsDict[zoneVal][validKLSCSum] += KLSC
                statsDict[zoneVal][validKLSCFinesSum] += KLSCFines
                statsDict[zoneVal][validDataCountStr] += 1
                statsDict[zoneVal][validCFactSum] += theC
            
            
            
            colid += 1
        
        
        rowid += 1
        
        
    #now work out which stats to use
    #look to see if we'll use the default Cfactor derived values in lieu of valid entries
    for uid in statsDict.keys():
        
        #if uid == 34:
        #    print(statsDict[uid])
        
        zoneCount = statsDict[uid][zoneCountStr]
        if zoneCount == 0:
            #nothing to report, add zeros
            statsDictToReturn[uid] = {KLSCValStr: -9999, KLSCFinesValStr: -9999, CFactValStr: -9999}
            continue
        
        valProp = 0
        valCount = statsDict[uid][validDataCountStr]
        if(valCount > 0):
            valProp = valCount / zoneCount
         
        if valProp >= theReqValidProp:
            # use valid only stats
            statsDictToReturn[uid] = {KLSCValStr: float(statsDict[uid][validKLSCSum] / valCount),
                                      KLSCFinesValStr: float(statsDict[uid][validKLSCFinesSum] / valCount),
                                      CFactValStr: float(statsDict[uid][validCFactSum] / valCount)}
        else:
            # Use stats that include the default where necessary
            statsDictToReturn[uid] = {KLSCValStr: float(statsDict[uid][allKLSCSum] / zoneCount),
                                      KLSCFinesValStr: float(statsDict[uid][allKLSCFinesSum] / zoneCount),
                                      CFactValStr: float(statsDict[uid][allCFactSum] / zoneCount)}
        
    
    statsWithDate = {}
    statsWithDate[theRepDate] = statsDictToReturn
    
    theCFatcorRas = None
    theZonesRasterArray = None
    zonesGT = None
    ZonesRowCount =  None
    ZonesColCount = None
    ZoneND = None
    theDefaultC = None
    theReqValidProp = None
    theKLSNumpy = None
    theKLSFinesNumpy = None
    
    gc.collect()
    
    return statsWithDate

In [107]:
#start_time = time.time()


##tester = analyseThisCFactorRaster(r"D:\\MW\\Ram_USLE\\skip\\justone\\cf_198712_198802.tif", 
##                                  zonesRasterArray, zone_ds.GetGeoTransform(), 
##                                           zone_ds.RasterYSize, zone_ds.RasterXSize, NoData_value, defaultCfact, 
##                                           reqdValidProp, KLSNumpyFilled, KLSFinesNumpyFilled)

#tester = analyseThisCFactorRaster(r"C:\DDrive\MW\v450projs\Ram\c-factors\cf_201809_201811.tif", 
#                                  cFactPattern, zonesRasterArray, zone_ds.GetGeoTransform(), 
#                                           zone_ds.RasterYSize, zone_ds.RasterXSize, NoData_value, defaultCfact, 
#                                           reqdValidProp, KLSNumpyFilled, KLSFinesNumpyFilled)

#elapsed_time = time.time() - start_time
#print("Finished One: " + str(len(tester)) + " taking this many seconds: " + str(elapsed_time))

In [108]:
#print(tester)

In [109]:
#%load usle_cfactor_analysis

start_time = time.time()

usleStats = {}

mydict=dict(theZonesRasterArray=zonesRasterArray, datePattern=cFactPattern, zonesGT=zone_ds.GetGeoTransform(), ZonesRowCount=zone_ds.RasterYSize, 
            ZonesColCount=zone_ds.RasterXSize, ZoneND=NoData_value, 
            theDefaultC=defaultCfact, theReqValidProp=reqdValidProp, 
            theKLSNumpy=KLSNumpyFilled, theKLSFinesNumpy=KLSFinesNumpyFilled,
            paramANumpy=paramAFilled, paramBNumpy=paramBFilled, localPerformChange=performChange)


#print(mydict)

#num_cores = multiprocessing.cpu_count()
#print(str(num_cores) + " cores available")

#xxx = lambda cRas:usle_cfactor_analysis.analyseThisCFactorRaster(cRas,
#                                                    zonesRasterArray, zone_ds.RasterYSize, 
#                                                    zone_ds.RasterXSize, NoData_value, 
#                                                    defaultCfact, reqdValidProp)

#xxx = lambda cRas:analyseThisCFactorRaster(cRas, zonesRasterArray, zone_ds.GetGeoTransform(), 
#                                           zone_ds.RasterYSize, zone_ds.RasterXSize, 
#                                           NoData_value, defaultCfact, reqdValidProp)

xxx = lambda theCFactorRas:analyseThisCFactorRaster(theCFactorRas, datePattern, theZonesRasterArray, zonesGT, 
                                           ZonesRowCount, ZonesColCount, ZoneND, theDefaultC, 
                                           theReqValidProp, theKLSNumpy, theKLSFinesNumpy,
                                            paramANumpy, paramBNumpy, localPerformChange)


#xxx = lambda rasDate:usle_cfactor_analysis.analyseThisCFactorRaster(cRastersInfo.get(rasDate),
#                                                    zonesRasterArray, zone_ds.RasterYSize, 
#                                                    zone_ds.RasterXSize, NoData_value, 
#                                                    defaultCfact, reqdValidProp)

print("doing execute")
#rc[:].execute('import D:/PythonScriptingForGBR/reef_scripts/GBR/Examples/Modules/usle_cfactor_analysis')
rc[:].execute("import numpy")
rc[:].execute("import gc")
rc[:].execute("import os")
rc[:].execute("import datetime")
rc[:].execute("from osgeo import gdal, gdalconst")
rc[:].execute("from osgeo import ogr")

print("doing push")
rc[:].push({"analyseThisCFactorRaster":analyseThisCFactorRaster})
rc[:].push({"centreCoordsForPixel":centreCoordsForPixel})
rc[:].push({"pixelForCentreCoords":pixelForCentreCoords})
rc[:].push({"getDateFromFileName":getDateFromFileName})
print("starting pushing dict")
rc[:].push(mydict)
print("finished pushing dict")
results = rc[:].map_sync(xxx, cRastersAlone)
#results = rc[:].map_sync(xxx, cRastersInfoKeysSorted)

#results = rc[:].map_sync(lambda rasDate: usle_cfactor_analysis.analyseThisCFactorRaster(cRastersInfo.get(rasDate),
#                                                    zonesRasterArray, zone_ds.RasterYSize, 
#                                                    zone_ds.RasterXSize, NoData_value, 
#                                                    defaultCfact, reqdValidProp), 
#                         cRastersInfoKeysSorted)




#results = Parallel(n_jobs=num_cores)(delayed(analyseThisCFactorRaster)(cRastersInfo.get(rasDate), zonesRasterArray, zone_ds.RasterYSize, zone_ds.RasterXSize, NoData_value, defaultCfact, reqdValidProp) 
#                                     for rasDate in cRastersInfoKeysSorted)



#for rasDate in cRastersInfoKeysSorted:
    #Process the cFator raster
    # processedStats will have the date as primary key and then each Unique FU ID and some relevant stats
#    start_time = time.time()
#    print("starting work on " + str(cRastersInfo.get(rasDate)))
    
#    processedStats = analyseThisCFactorRaster(cRastersInfo.get(rasDate))
    
#    elapsed_time = time.time() - start_time
    
#    print("finished work on " + str(cRastersInfo.get(rasDate)) + " taking " + str(elapsed_time) + " seconds")
#    print("this big " + str(len(processedStats)))
    
    
#    #Add results to the stats
#    usleStats[rasDate] = processedStats
    
elapsed_time = time.time() - start_time

print("Got this many rasters processed: " + str(len(results)) + " taking this many seconds: " + str(elapsed_time))

doing execute
doing push
starting pushing dict
finished pushing dict
Got this many rasters processed: 129 taking this many seconds: 10039.44669508934


In [110]:
#type(results[0])
#results[0].keys()
convertedDict = {}
availDates = []
allConvertedToDict = {}
for gg in results:
    #print(str(gg.keys()) + " and " + str(len(gg.keys())))
    #Know that we only have one key, but need to loop anyway
    for dateKey in gg.keys():
        convertedDict[dateKey] = gg.values()
        availDates.append(dateKey)
        allConvertedToDict[dateKey] = {}
        
        for dataPairs in gg.values():
            for fuKeys in dataPairs.keys():
                #print(fuKeys)
                allConvertedToDict[dateKey][fuKeys] = dataPairs[fuKeys]
            #print(dataPairs.values())
    #convertedDict[gg.keys()[0]] = gg.values()


print(len(convertedDict))
sortedAvailDates = sorted(availDates)
#convertedDict
#print(convertedDict[dateKey])

129


In [111]:
#convertedDict[datetime.date(1988, 3, 1)]
#convertedDict.keys()

In [112]:
#print(allConvertedToDict)

In [113]:
# now need FU-based KLSC & KLSCFines averages for tempral gap filling
avgs = {}
unZones = numpy.unique(zonesRasterArray)
klscSumStr = "KLSCSum"
klscfinesSumStr = "KLSCFinesSum"
cfactSumStr = "CFactSum"
klscCountStr = "KLSCCount"
klscfinesCountStr = "KLSCFinesCount"
cfactCountStr = "CFactCount"

KLSCGlobalSum = 0
KLSCGlobalCount = 0
KLSCFinesGlobalSum = 0
KLSCFinesGlobalCount = 0
CFactGlobalSum = 0
CFactGlobalCount = 0


for zn in unZones:
    avgs[zn] = {klscSumStr: float(0), klscCountStr: int(0), 
                klscfinesSumStr: float(0), klscfinesCountStr: int(0), 
                cfactSumStr: float(0), cfactCountStr: int(0)}

# now populate
for dt in convertedDict.keys():
    #print(type(convertedDict[dt]))
    #print(convertedDict[dt])
    for fuDict in convertedDict[dt]:
        #print(type(fuDict))
        for fuid in fuDict.keys():
            #print(fuDict[fuid])

            if not fuDict[fuid][KLSCValStr] == -9999:
                avgs[fuid][klscSumStr] += fuDict[fuid][KLSCValStr]
                avgs[fuid][klscCountStr] += 1
                KLSCGlobalSum += fuDict[fuid][KLSCValStr]
                KLSCGlobalCount += 1
            if not fuDict[fuid][KLSCFinesValStr] == -9999:
                avgs[fuid][klscfinesSumStr] += fuDict[fuid][KLSCFinesValStr]
                avgs[fuid][klscfinesCountStr] += 1
                KLSCFinesGlobalSum += fuDict[fuid][KLSCFinesValStr]
                KLSCFinesGlobalCount += 1
            if not fuDict[fuid][CFactValStr] == -9999:
                avgs[fuid][cfactSumStr] += fuDict[fuid][CFactValStr]
                avgs[fuid][cfactCountStr] += 1
                CFactGlobalSum += fuDict[fuid][CFactValStr]
                CFactGlobalCount += 1

#print(avgs[3])
#print(avgs[36])
#print(avgs[1397])
KLSCGlobalAvg = 0
KLSCFinesGlobalAvg = 0
CFactGlobalAvg = 0

if KLSCGlobalCount > 0:
    KLSCGlobalAvg = KLSCGlobalSum / KLSCGlobalCount

if KLSCFinesGlobalCount > 0:
    KLSCFinesGlobalAvg = KLSCFinesGlobalSum / KLSCFinesGlobalCount

if CFactGlobalCount > 0:
    CFactGlobalAvg = CFactGlobalSum / CFactGlobalCount

In [114]:
#print(len(avgs))
#print(avgs[3])
#print(avgs[36])
#print(avgs[1397])

In [115]:
# would now need to determine the required timespan for output timeseries
# grabbing relevant values form either real or temporal average as needed

reqdStart = datetime.date(1986, 1, 1)
reqdLast = datetime.date(2018, 12, 31)

earliestAvail = sortedAvailDates[0]
latestAvail = sortedAvailDates[len(sortedAvailDates) - 1]

print(earliestAvail)
print(latestAvail)

1986-06-01
2018-06-01


In [116]:
if reqdStart < earliestAvail:
    print("Defs need padding at start")
else:
    print("No start padding required")

Defs need padding at start


In [117]:
if reqdLast > latestAvail:
    print("Defs need padding at end")
else:
    print("No end padding required")

Defs need padding at end


In [118]:
###I think I would then create a full list of required dates (varying with daily, monthly, seasonal, annual etc)
###And then create a CSV writing loop based on indivudal FUs, where once processing a FU we begin stepping through
### the full dates, looking for existence in convertedDict (taken from 'results')... although it has DATE as first key...
###hmmm


from dateutil.relativedelta import relativedelta
import calendar

bollocks = reqdStart + relativedelta(months=1)

print(bollocks)

bollocks2 = reqdLast + relativedelta(months=1)

print(bollocks2)

print(reqdLast.year)
print(reqdLast.month)
print(reqdLast.day)

### Ok, manipulate start & end dates to 1st of month
actualStartDate = datetime.date(reqdStart.year, reqdStart.month, 1)
actualEndDate = datetime.date(reqdLast.year, reqdLast.month, calendar.monthrange(reqdLast.year, reqdLast.month)[1])
print(actualStartDate)
print(actualEndDate)

theDates = []

assessDate = actualStartDate

while assessDate < actualEndDate:
    theDates.append(assessDate)
    assessDate = assessDate + relativedelta(months=1)


#print(theDates)

1986-02-01
2019-01-31
2018
12
31
1986-01-01
2018-12-31


In [119]:
###Now try to get at the values... use FU = 1 to start....

#print(fumapper)
#"'" + str(theFUID) + "'"

#theFUID = 1
#theRow = fumapper.loc[fumapper[uniqueIDField] == str(theFUID)]
#print(theRow)
#theFuFileNameBit = str(theRow[catchmentField].values[0]) + "$" + str(theRow[FUField].values[0])

#print(theFuFileNameBit)

In [120]:
#listDateKeys = list(convertedDict.keys())
##print(listTest)
#print(listTest[3])

In [121]:
lastRealDate = theDates[-1]
#print(lastRealDate)
#print(unZones)
for zn in unZones:
    
    if zn < 0:
        continue
    
    theFUID = zn
    #print(str(theFUID))
    theRow = fumapper.loc[fumapper[uniqueIDField] == str(theFUID)]
    #print(theRow)
    theFuFileNameBit = str(theRow[catchmentField].values[0]) + "$" + str(theRow[FUField].values[0])
    
    klscName = os.path.join(parentCSVOutputDir, 'KLSC', theFuFileNameBit + '$USLE_KLSC_Total.csv')
    klscFinesName = os.path.join(parentCSVOutputDir, 'KLSC_Fines', theFuFileNameBit + '$USLE_KLSC_FinePerc.csv')
    CfactName = os.path.join(parentCSVOutputDir, 'Cfact', theFuFileNameBit + '$CFactor.csv')

    klscFile = open(klscName,'w')
    klscFinesFile = open(klscFinesName,'w')
    CfactFile = open(CfactName,'w')

    klscFile.write('Date,KLSC_Total' + '\n')
    klscFinesFile.write('Date,KLSC_Fines' + '\n')
    CfactFile.write('Date,C-Factor' + '\n')

    
    
    for thisDate in theDates:
        theSep = '\n'
        ##Convert to seaonal date of relevance
        seasonalDate = thisDate + relativedelta(months=monthToSeasonOffsets[thisDate.month])
        if seasonalDate in allConvertedToDict.keys():
            #print("In Here")
            #klscFile.write(str(thisDate) + theSep)
            #print(allConvertedToDict[seasonalDate])
            if theFUID in allConvertedToDict[seasonalDate]:
                klscFile.write(str(thisDate) + ',' + str(allConvertedToDict[seasonalDate][theFUID][KLSCValStr]) + theSep)
                klscFinesFile.write(str(thisDate) + ',' + str(allConvertedToDict[seasonalDate][theFUID][KLSCFinesValStr]) + theSep)
                CfactFile.write(str(thisDate) + ',' + str(allConvertedToDict[seasonalDate][theFUID][CFactValStr]) + theSep)
            else:
                #Add in average
                if theFUID in avgs:
                    klscFile.write(str(thisDate) + ',' + str((avgs[theFUID][klscSumStr]/avgs[theFUID][klscCountStr])) + theSep)
                    klscFinesFile.write(str(thisDate) + ',' + str((avgs[theFUID][klscfinesSumStr]/avgs[theFUID][klscfinesCountStr])) + theSep)
                    CfactFile.write(str(thisDate) + ',' + str((avgs[theFUID][cfactSumStr]/avgs[theFUID][cfactCountStr])) + theSep)
                else:
                    #Global avg???
                    klscFile.write(str(thisDate) + ',' + str(KLSCGlobalAvg) + theSep)
                    klscFinesFile.write(str(thisDate) + ',' + str(KLSCFinesGlobalAvg) + theSep)
                    CfactFile.write(str(thisDate) + ',' + str(CFactGlobalAvg) + theSep)
        else:
            #Add in average
            if theFUID in avgs:
                klscFile.write(str(thisDate) + ',' + str((avgs[theFUID][klscSumStr]/avgs[theFUID][klscCountStr])) + theSep)
                klscFinesFile.write(str(thisDate) + ',' + str((avgs[theFUID][klscfinesSumStr]/avgs[theFUID][klscfinesCountStr])) + theSep)
                CfactFile.write(str(thisDate) + ',' + str((avgs[theFUID][cfactSumStr]/avgs[theFUID][cfactCountStr])) + theSep)
            else:
                #Global avg???
                klscFile.write(str(thisDate) + ',' + str(KLSCGlobalAvg) + theSep)
                klscFinesFile.write(str(thisDate) + ',' + str(KLSCFinesGlobalAvg) + theSep)
                CfactFile.write(str(thisDate) + ',' + str(CFactGlobalAvg) + theSep)

    klscFile.close()
    klscFinesFile.close()
    CfactFile.close()

print("Done")

Done
